# Machine learning, FCS HSE

## Practical task 4. Decomposition of error into bias and spread


### About the task

In this assignment, you will use the power of bootstrap to estimate the bias and scatter of machine learning algorithms. We will do this on the boston data:

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
from sklearn.datasets import load_boston

In [3]:
boston = load_boston()

C:\Users\solmi\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this case special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing data

In [4]:
X = boston["data"]
y = boston["target"]

In [5]:
X.shape, y.shape

((506, 13), (506,))

### Computing bias and variance with bootstrap
At the lecture, the following formula was derived, showing how the error of the regression algorithm can be represented as the sum of three components:
$$
L(\mu) =
    \mathbb{E}_{x, y}\bigl[\mathbb{E}_{X}\bigl[ (y - \mu(X)(x))^2 \bigr]\bigr] =
$$
$$
    \underbrace{\mathbb{E}_{x, y}\bigl[(y - \mathbb{E}[y|x] )^2\bigr]}_{\text{noise)) + \underbrace{\ mathbb{E}_{x}\bigl[(\mathbb{E}_{X}[\mu(X)(x)] - \mathbb{E}[y|x] )^2\bigr]}_ {\text{offset}} +
    \underbrace{\mathbb{E}_{x}\bigl[\mathbb{E}_{X}\bigl[(\mu(X)(x) - \mathbb{E}_{X}[\mu( X)(x)] )^2\bigr]\bigr]}_{\text{scatter)),
$$
* $\mu(X)$ is an algorithm trained on the sample $X = \{(x_1, y_1), \dots (x_\ell, y_\ell)\}$;
* $\mu(X)(x)$ is the response of the algorithm trained on the sample $X$ on the object $x$;
* $\mathbb{E}_{X}$ — mat. expectation over all possible samples;
* $\mathbb{E}_{X}[\mu(X)(x)]$ is the "average" response of the algorithm trained on all possible samples $X$ on object $x$.
    
Using this formula, we can analyze the properties of the learning algorithm for the $\mu$ model if we specify a probabilistic model for generating $p(x, y)$ pairs.

In real problems, of course, we do not know the distribution on object pairs - the correct answer. However, we have a set of samples from this distribution (the training set) and we can use it to estimate the expected values. To evaluate mat. expectations on samples, we will use bootstrap - a method of generating "new" samples from one by selecting objects with a return. Let's take a look at a few steps on the way to estimating bias and scatter.

#### Approximate calculation of integrals
In the classroom, we analyzed examples of analytical calculation of bias and scatter of several learning algorithms. For most data models and learning algorithms, it will not be possible to analytically calculate mathematical expectations in formulas. However, mat. expectations can be approximated. To estimate the expectation $\mathbb{E}_{\bar z} f(\bar z)$ of a function of a multidimensional random variable $\bar z = (z_1, \dots, z_d)$, $\bar z \sim p (\bar z)$, you can generate a sample from the distribution $p(\bar z)$ and average the value of the function over the elements of this sample:
$$\mathbb{E}_{\bar z} f(z) = \int f(\bar z) p(\bar z) d \bar z \approx \frac 1 m \sum_{i=1}^ m f(\bar z_i), \, \bar z_i \sim p(\bar z), i = 1, \dots, m.$$

For example, let's estimate $\mathbb{E}_z z^2,$ $z \sim \mathcal{N}(\mu=5, \sigma=3)$ (we know from probability theory that
$\mathbb{E}_z z^2 = \sigma^2 + \mu^2 = 34$):

In [6]:
z = np.random.normal(loc=5, scale=3, size=10000)
(z**2).mean()

34.11884393391007

#### Estimating $\mathbb{E}_{x, y}$
Rate mat. the expectations on $x$ and on $x, y$ occurring in all three components of the decomposition is not difficult, because we have a sample of objects from the data distribution $p(x, y)$:
$$ \mathbb{E}_{x} f(x) \approx \frac 1 N \sum_{i=1}^N f(x_i), \quad
\mathbb{E}_{x, y} f(x, y) \approx \frac 1 N \sum_{i=1}^N f(x_i, y_i),$$
where $N$ is the number of objects in the sample, $\{(x_i, y_i)\}_{i=1}^N$ is the sample itself.

#### Estimating $\mathbb{E}_X$ with bootstrap
To evaluate mat. expectation on $X$, we need a sample from the samples:
$$\mathbb{E}_X f(X) \approx \frac 1 s \sum_{j=1}^s f(X_j),$$
where $X_j$ is the $j$th sample. To get them, we can use bootstrap - a method of generating selections based on a selection of objects with a return. To make one sample, we will select the object index $i \sim \text{Uniform}(1 \dots N)$ $N$ times and add the $i$-th pair (object, target variable) to the sample. As a result, duplicate objects may appear in each sample, and some objects may not be included in some samples at all.

#### The final algorithm for estimating the bias and scatter of the $a$ algorithm
1. Generate $s$ samples $X_j$ using the bootstrap method.
1. On each sample $X_j$, train the algorithm $a_j$.
1. For each sample $X_j$ determine the set of objects $T_j$ that are not included in it (out-of-bag). Calculate predictions of $a_j$ algorithm on $T_j$ objects.

Since we only have one answer for each object, we will assume that the noise is 0 and $\mathbb{E}[y|x]$ is equal to the correct answer for the object $x$.

Final grades:
* Offset: for one object, the square of the difference between the average prediction and the correct answer. The average prediction is taken only for those algorithms $a_j$ for which this object was included in the out-of-bag sample $T_j$. To obtain the total displacement, average the displacements over the objects.
* Scatter: for one object - the sample variance of $a_j$ algorithm predictions for which this object was included in the $T_j$ out-of-bag sample. To obtain a general scatter, average the scatter over objects.
* Error $L$: average the squared differences between the prediction and the correct answer over all predictions made for all objects.

As a result, it should turn out that the error is approximately equal to the sum of the bias and spread!

The algorithm is also briefly described at [link](https://web.engr.oregonstate.edu/~tgd/classes/534/slides/part9.pdf) (slides 19-21).

__one. (3 points)__

Implement the described algorithm. Please note that if the object is not included in any of the out-of-bag selections, you do not need to take it into account in calculating the total values. As usual, only one run is allowed - over samples (from 0 to num_runs-1).

In [7]:
def compute_biase_variance(regressor, X, y, num_runs=1000):
    """
    :param regressor: sklearn estimator with fit(...) and predict(...) method
    :param X: numpy-array representing training set ob objects, shape [n_obj, n_feat]
    :param y: numpy-array representing target for training objects, shape [n_obj]
    :param num_runs: int, number of samples (s in the description of the algorithm)
    
    :returns: bias (float), variance (float), error (float) 
    each value is computed using bootstrap
    """
    ind = np.random.randint(0, len(X), len(X)*num_runs).reshape(num_runs, -1)
    al = [x for x in range(len(X))]
    df = pd.DataFrame(columns=[0,1])
    error_all = np.array([])
    
    for i in ind:
        regressor.fit(X[i], y[i])
        ind_test = list(set(al) - set(i))
        y_pred = regressor.predict(X[ind_test])
        df = df.append(pd.DataFrame(list(zip(ind_test, y_pred))))
        error_all = np.append(error_all, (y[ind_test] - y_pred)**2)
    
    df.set_index(np.array([x for x in range(len(df))]), inplace = True)
    bias = np.mean((np.array(df.groupby(0).mean()).reshape(len(y),) - y)**2)
    variance = np.mean((np.array(df.groupby(0).std()))**2)
    error = error_all.mean()
    return bias, variance, error    


__2. (0 points)__

Estimate bias, scatter, and error for three algorithms with default hyperparameters: linear regression, decision tree, and random forest.

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

regressor = [LinearRegression(), DecisionTreeRegressor(), RandomForestRegressor(), GradientBoostingRegressor()]

for regressor in regressor:
    bias, variance, error = compute_biase_variance(regressor, X, y)
    print('{}: bias = {}; variance = {}; mse = {}'.format(regressor, bias, variance, error))

LinearRegression(): bias = 23.767655069714465; variance = 0.9554026796952855; mse = 24.803334108748416
DecisionTreeRegressor(): bias = 10.1140784639303; variance = 13.068350385077718; mse = 23.111529356571257
RandomForestRegressor(): bias = 10.75976069652386; variance = 2.2840408944566297; mse = 13.143811678552384
GradientBoostingRegressor(): bias = 9.210643476074777; variance = 2.031754526234437; mse = 11.223787739015004


In [8]:
from sklearn.model_selection import train_test_split
from mlxtend.evaluate import bias_variance_decomp

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
regressor = [LinearRegression(), DecisionTreeRegressor(), RandomForestRegressor(), GradientBoostingRegressor()]

for regressor in regressor:
    mse, bias, variance = bias_variance_decomp(regressor, X_train, y_train, X_test, y_test, loss='mse', num_rounds=200, random_seed=1)
    print('{}: bias = {}; variance = {}; mse = {}'.format(regressor, bias, variance, error))

LinearRegression(): bias = 32.43604736455349; variance = 1.6408175036848205; mse = 11.037919652173743
DecisionTreeRegressor(): bias = 16.53716414215689; variance = 16.079058406862746; mse = 11.037919652173743
RandomForestRegressor(): bias = 16.928774081232838; variance = 3.1936187681789248; mse = 11.037919652173743
GradientBoostingRegressor(): bias = 13.356529621428038; variance = 3.414108077099948; mse = 11.037919652173743
